# Hadoop Streaming : Stop Words
Calculate how many stop words are in the input dataset. Stop words list is in ‘/datasets/stop_words_en.txt’ file. Use Hadoop counter to count the number of stop words and total words in the dataset. The result is the percentage of stop words in the entire dataset (without percent symbol).

In [153]:
%%writefile mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

with open('stop_words_en.txt') as f:
    stop_words = set(f.read().split())
    
for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        if word in stop_words:
            print >> sys.stderr, "reporter:counter:Wiki stats,Stop words,%d" % 1
        print "%s\t%d" % (word.lower(), 1)


Overwriting mapper.py


In [154]:
%%writefile counter_process.py
import sys
import re

stop_words=0
all_words=0
while
stop_words = int(re.search('\d+|$', sys.stdin.readline()).group())
all_words = int(re.search('\d+|$', sys.stdin.readline()).group())
print(stop_words/float(all_words)*100)



Overwriting counter_process.py


In [155]:
! hdfs dfs -ls /data/wiki

Found 1 items
drwxrwxrwx   - jovyan supergroup          0 2017-10-17 13:15 /data/wiki/en_articles_part


In [156]:
%%bash

OUT_DIR="wordcount_result_"$(date +"%s%6N")
NUM_REDUCERS=0

LOGS="stderr_logs.txt"

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming stopwords" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,/datasets/stop_words_en.txt \
    -mapper "python mapper.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null 2> $LOGS


rm: `wordcount_result_1525929437201909': No such file or directory


In [168]:
%%bash
LOGS="stderr_logs.txt"
cat $LOGS |egrep "*words=([0-9]+)"| python ./counter_process.py
cat $LOGS >&2

38.44036900909957


18/05/10 05:17:20 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/05/10 05:17:21 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/05/10 05:17:21 INFO mapred.FileInputFormat: Total input files to process : 1
18/05/10 05:17:21 INFO mapreduce.JobSubmitter: number of splits:2
18/05/10 05:17:22 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1525903343186_0015
18/05/10 05:17:22 INFO impl.YarnClientImpl: Submitted application application_1525903343186_0015
18/05/10 05:17:22 INFO mapreduce.Job: The url to track the job: http://cbccbb36aa92:8088/proxy/application_1525903343186_0015/
18/05/10 05:17:22 INFO mapreduce.Job: Running job: job_1525903343186_0015
18/05/10 05:17:28 INFO mapreduce.Job: Job job_1525903343186_0015 running in uber mode : false
18/05/10 05:17:28 INFO mapreduce.Job:  map 0% reduce 0%
18/05/10 05:17:44 INFO mapreduce.Job:  map 36% reduce 0%
18/05/10 05:17:50 INFO mapreduce.Job:  map 54% reduce 0%
18/05/10 05:17:56 INFO 